In [2]:
%matplotlib inline

import os
import datacube
import warnings
import pydotplus
import numpy as np
import geopandas as gpd
from io import StringIO
from joblib import dump
from pprint import pprint
import matplotlib.pyplot as plt
from IPython.display import Image
from odc.io.cgroups import get_cpu_quota
from sklearn import tree, model_selection
from sklearn.metrics import accuracy_score
from osgeo import gdal
from datacube.utils.cog import write_cog
# from reo_cogeo.cogeo import translate_cog

from deafrica_tools.plotting import map_shapefile, rgb
from deafrica_tools.classification import predict_xr, collect_training_data
from deafrica_tools.bandindices import calculate_indices

In [3]:
warnings.filterwarnings('ignore')

In [4]:
dc = datacube.Datacube(app='ML_with_ODC')

In [1]:
# lat_range = (15.75418332, 15.85828652)
# lon_range = (80.78694696, 81.02203692)
# time_range = ('2022-01-15', '2023-02-15')
# # display_map(x=lon_range, y=lat_range)
# ds = dc.load(product="s2a_sen2cor_granule",
#              x=lon_range,
#              y=lat_range,
#              time=time_range,
#              output_crs='EPSG:6933',
#              resolution=(-30, 30))
# output_cog = 's2_with_bcmad.tif'
# translate_cog(ds, output_cog, overwrite=True)
# import rasterio

# # Open Sentinel-2 image and read metadata
# with rasterio.open('sentinel2.tif') as src:
#     profile = src.profile
#     image = src.read()

# # Compute BCMAD band
# b, c, m, a, d = image[1], image[2], image[3], image[6], image[7]
# bc = b - c
# am = a - m
# bc_ad = bc - am
# bcmad = bc_ad / (bc + am)

# # Update metadata to include BCMAD band
# profile.update(count=3)

# # Write new image with BCMAD band
# with rasterio.open('sentinel2_bcmad.tif', 'w', **profile) as dst:
#     dst.write(image[:4], 1)
#     dst.write(bcmad, 4)
#     dst.write(image[4:], 5)



# import os
# from osgeo import gdal

# # set the output file name and format
# out_vrt = './sentinel2.vrt'
# out_tif = './sentinel2.tif'
# gdalformat = 'GTiff'

# # set the resolution in meters
# xres = 10
# yres = -10

# # set the common projection and extent
# projection = 'EPSG:4326'
# xmin, ymin, xmax, ymax = (80.78694696, 15.75418332, 81.02203692, 15.85828652) # set the extent of your area of interest

# # get a list of all the input bands
# temp1 = './S2A_MSIL2A_20230209T045941_N0509_R119_T44PMC_20230209T085054.SAFE/GRANULE/L2A_T44PMC_A039870_20230209T050547/IMG_DATA/R10m'
# # temp2 = './S2A_MSIL2A_20230209T045941_N0509_R119_T44PMC_20230209T085054.SAFE/GRANULE/L2A_T44PMC_A039870_20230209T050547/IMG_DATA/R20m'
# input_bands = [f'{temp1}/T44PMC_20230209T045941_B02_10m.jp2', f'{temp1}/T44PMC_20230209T045941_B03_10m.jp2', f'{temp1}/T44PMC_20230209T045941_B04_10m.jp2', f'{temp1}/T44PMC_20230209T045941_B08_10m.jp2']

# # resample all the bands to a common resolution
# resampled_bands = []
# for band in input_bands:
#     # set the output band file name
#     out_band = os.path.splitext(band)[0] + '_resampled.jp2'
    
#     # resample the band
#     os.system(f"gdalwarp -r cubic -t_srs {projection} -te {xmin} {ymin} {xmax} {ymax} -tr {xres} {yres} {band} {out_band}")
    
#     resampled_bands.append(out_band)

# # create a virtual raster file (.vrt) to stack all the bands
# os.system(f"gdalbuildvrt -separate {out_vrt} {' '.join(resampled_bands)}")

# # convert the virtual raster file to a GeoTIFF file
# os.system(f"gdal_translate -of {gdalformat} {out_vrt} {out_tif}")



In [8]:
lat, long = 15.80, 80.90 # Kampala, Uganda
buffer = 0.15
path = '../clipped shape file/clipped_mangroves.shp'
field = 'PXLVAL'
products = 'landsat_8_c2_l2'
measurements =  ['blue','green','red','B08_10m','B11_20m','B12_20m', 'BCMAD', 'EMAD', 'SMAD']
time = ('2022')
zonal_stats = None #'median'
resolution =  (-30, 30)
# ds = calculate_indices(ds, index='BCMAD', collection='sentinel_2')

In [6]:
ncpus=get_cpu_quota()
print('ncpus = '+str(ncpus))

ncpus = None


In [9]:
#open shapefile and ensure its in WGS84 coordinates
input_data = gpd.read_file(path).to_crs('epsg:4326')

#check the shapfile by plotting it
map_shapefile(input_data, attribute=field)

Label(value='')

Map(center=[15.966111111111646, 80.89622222222184], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [8]:
def feature_function(query):
    dc = datacube.Datacube(app='feature_layers')
    ds = dc.load(**query)
    ds = ds.mean('time')
    return ds

In [9]:
def feature_layers(query):
    #connect to the datacube
    dc = datacube.Datacube(app='feature_layers')

    #load ls8 geomedian
    ds = dc.load(product=products,
                 **query)

    #calculate some band indices
    ds = calculate_indices(ds,
                           index=['NDVI', 'BUI', 'MNDWI'],
                           drop=False,
                           satellite_mission='s2')

    return ds

In [10]:
# import rasterio

# # Load Sentinel-2 bands 6, 7, and 8
# temp1 = './S2A_MSIL2A_20230209T045941_N0509_R119_T44PMC_20230209T085054.SAFE/GRANULE/L2A_T44PMC_A039870_20230209T050547/IMG_DATA/R20m'
# with rasterio.open(f'{temp1}/B06_20m.jp2') as b6:
#     B6 = b6.read(1)
# with rasterio.open(f'{temp1}/B07_20m.jp2') as b7:
#     B7 = b7.read(1)
# with rasterio.open(f'{temp1}/B08_20m.jp2') as b8:
#     B8 = b8.read(1)

# # Calculate BCMAD
# BCMAD = (1 - (B8 / B7)) * (1 - (B6 / B7)) * (B8 / B6)

# # Save BCMAD to a new GeoTIFF file
# with rasterio.open('path/to/BCMAD.tif', 'w', 
#                    driver='GTiff', 
#                    height=BCMAD.shape[0], 
#                    width=BCMAD.shape[1], 
#                    count=1, 
#                    dtype=BCMAD.dtype, 
#                    crs=b6.crs, 
#                    transform=b6.transform) as dst:
#     dst.write(BCMAD, 1)

In [11]:
#generate a datacube query object
query = {
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'output_crs' : 'epsg:6933'
}

In [12]:
# gdf['geometry'] = gdf['geometry'].astype(int)
column_names, model_input = collect_training_data(
                                    gdf=input_data,
                                    dc_query=query,
                                    field=field,
                                    zonal_stats=zonal_stats,
                                    feature_func=feature_layers
                                    )

ValueError: Please verify that all bands required to compute NDVI are present in `ds`.